# Setup del proyecto

En esta sección preparamos el entorno de trabajo necesario para el proyecto.

## Instalaciones

In [3]:
%pip install python-dotenv pandas

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 13.2 MB/s  0:00:003.0 MB/s eta 0:00:0101
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 26.6 MB/s  0:00:00
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pandas]━━━━ 3/4 [pandas]
Note: you may need to restart the kernel to use updated packages.


## importaciones

In [2]:
from dotenv import load_dotenv
import pandas as pd
from pathlib import Path
import json
import os 
import pprint
import sqlite3

## Crear la BBDD

In [7]:
import sqlite3

def generate_sqlite_database(sql_script, sqlite_db):

    # 1. Connect to the SQLite database
    conn = sqlite3.connect(sqlite_db)

    try:
        # 2. Read the SQL script from a file
        with open(sql_script, 'r') as f:
            sql_script = f.read()

        # 3. Execute the SQL script
        conn.executescript(sql_script)
        print("SQL script executed successfully.")

        # 4. Commit changes
        conn.commit()

    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
        # Rollback changes in case of an error
        conn.rollback()

    finally:
        # 5. Close the connection
        conn.close()

generate_sqlite_database('./data/create_db.sql', './data/consultorio.db')  

SQL script executed successfully.


## Carga de datos a BBDD
Carga de datos a la BBDD desde los archivos de datos fuente

### Pacientes

In [10]:
# Obtenemos los archivos de datos fuente
df_pacientes = pd.read_csv('./data/pacientes_ficticios.csv')
df_evoluciones = pd.read_csv('./data/evoluciones.csv')


In [12]:
def insert_data_to_db(df_pacientes, df_evoluciones, db_name):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Insert pacientes
    for _, row in df_pacientes.iterrows():
        cursor.execute("""
            INSERT INTO paciente (id_paciente, apellido, nombre, fecha_nac, sexo, edad, dni, localidad, obra_social, afiliado_nro, telefono, telefono_numero, email, especialidad, diagnostico, enfermedad_base, ant_perinatales, ant_familiares, registro, fecha_registro)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            row['id_paciente'], row['apellido'], row['nombre'], row['fecha_nac'], row['sexo'], row['edad'], row['dni'], row['localidad'], row['obra_social'], row['afiliado_nro'], row['telefono'], row['telefono_numero'], row['email'], row['especialidad'], row['diagnostico'], row['enfermedad_base'], row['ant_perinatales'], row['ant_familiares'], row['registro'], row['fecha_reg']
        ))

    # Insert evoluciones
    for _, row in df_evoluciones.iterrows():
        cursor.execute("""
            INSERT INTO evolucion (id, id_paciente, fecha, edad, uni_edad, edad_anios, edad_texto, peso, talla, imc, pc, motivo, conducta)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            row['id'], row['id_paciente'], row['fecha'], row['edad'], row['uni_edad'], row['edad_anios'], row['edad_texto'], row['peso'], row['talla'], row['imc'], row['pc'], row['motivo'], row['conducta']
        ))

    conn.commit()
    conn.close()
    
insert_data_to_db(df_pacientes, df_evoluciones, './data/consultorio.db')

In [13]:
import pprint 

## cooroborar que los datos se subieron correctamente a la BBDD
def get_one_patient(db_name, patient_id):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    cursor.execute("SELECT * FROM paciente WHERE id_paciente = ?", (patient_id,))
    patient = cursor.fetchone()

    conn.close()
    return patient  

def get_one_evolution(db_name, evolution_id):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    cursor.execute("SELECT * FROM evolucion WHERE id = ?", (evolution_id,))
    evolution = cursor.fetchone()

    conn.close()
    return evolution

pprint.pprint(get_one_patient('./data/consultorio.db', 150))
pprint.pprint(get_one_evolution('./data/consultorio.db', 69))

(150,
 'Fernandez',
 'Lucas',
 '2012-02-06',
 'Varón',
 '13 Años',
 '52.045.238',
 'Olavarría',
 'TV SALUD',
 '09873480',
 '2284-591234',
 '2284591234',
 None,
 'Pediátrico',
 'Otra',
 'Klippel Trenaunay',
 'RNT PAEG vaginal Klippel Trenaunay Dx realizado al momento del nacer con uso '
 'de media de compresión',
 'No refiere',
 'Administrador',
 'Tue Apr 08 2025 10:39:10')
(69,
 323,
 '2025-08-14',
 0,
 None,
 1.31,
 '1 años y 4 meses',
 10.9,
 76.0,
 18.87,
 47.5,
 'Control de salud',
 'Peso 10.970')
